#### Интерактивные графики matplotlib

In [ ]:
import numpy as np
def Energy(p):
    kkk=1.6022e-12
    Te=5.e2
    me=9.1094e-28
    c=3.e10
    e=4.8032e-10
    bet2=kkk*Te/(me*c**2)
    gam = np.sqrt(1.0+p**2*bet2)
    E=(gam-1)*me*c**2/kkk/1000
    return E

In [ ]:
with open('distr.dat') as f:
    n1 = int(f.readline())
    n2 = int(f.readline())
    n3 = int(f.readline())
    print(n1,n2,n3)
    lines = f.readlines()
    data = []
    for li in lines:
        z = [float(v) for v in li.split()]
        data += z
print(len(data))
print(data[0:5])

In [ ]:
cs  = np.array(data[n1:n1+n2])
cs[n2-3]

In [ ]:
xp = np.array(data[0:n1])
Eng = Energy(xp)
csn = np.array(data[n1:n1+n2-2])
csn[n2-3] = 1
theta = np.arccos(csn)


In [ ]:
dff = np.array(data[n1+n2-2:n1+n2-2 + n1*(n2-2)])
df = np.resize(dff,(n2-2,n1))
print(df.shape)
print(n1*(n2-2))

log_df = np.log10(df)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x= xp, y=Eng),
    row=1, col=1
)

fig.add_trace(
    go.Scatter( y=csn),
    row=1, col=2
)

fig.update_layout(height=400, width=900, title_text="Eng (Kev)")
fig.show()

In [ ]:
xpg, csng = np.meshgrid(xp, csn)
Engg = Energy(xpg)
d = {'xp' : xpg.flatten(), 'csn': csng.flatten(),'Eng(KeV)': Engg.flatten(), 'df': dff}
import pandas as pd
df_df = pd.DataFrame(data=d)
df_df

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

#df = px.data.gapminder()
fig = px.line(df_df, x="Eng(KeV)", y="df",  animation_frame="csn",
           log_y=True)
fig.add_trace(
    go.Scatter( x = Eng, y=df[0,:],  name=f'cs= {csn[0]}' )
)
fig.add_trace(
    go.Scatter( x = Eng, y=df[500,:],  name=f'cs= {csn[500]}' )
)
fig.add_trace(
    go.Scatter( x = Eng, y=df[700,:],  name=f'cs= {csn[700]}' )
)
fig.add_trace(
    go.Scatter( x = Eng, y=df[-1,:],  name=f'cs= {csn[-1]}' )
)
fig.show()

In [ ]:
df_min, df_max = log_df.min(), log_df.max()
df_min, df_max

In [ ]:
import plotly.graph_objects as go
low = df_min/3
fig = go.Figure(data =
    go.Contour(
        z=log_df, x= Eng, y= csn,
        contours=dict(
            start= low, 
            end= df_max,
            size= (df_max-low)/30
        ),
    ))
fig.update_layout( 
      autosize=False,
      width=800,
      height=600,
      title="Contour of log(df(e, csn))",
      xaxis_title="Energy (KeV)",
      yaxis_title="cos(theta)",
      legend_title="Legend Title",
)


fig.show()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
# Read data from a csv

log_df[(log_df < -25.0)] = -25.0

fig = go.Figure(data=[go.Surface(z=log_df, x=Eng, y=theta)])
fig.update_layout( title='log df(eng, theta)', 
    scene = dict(xaxis_title='Energy (KeV)',
                 yaxis_title='cos(theta)',
                 zaxis_title='log(df)'),
    autosize=False,
    width=900, height=700,
    margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
x_max = max(xp)
x_max

In [ ]:
from scipy.interpolate import RegularGridInterpolator
interp_df = RegularGridInterpolator((theta, xp), log_df, bounds_error=False, fill_value=None)

In [ ]:
def cart2polNP(xy):
    x, y = xy
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(phi, rho)

x = np.linspace(-x_max, x_max, 1200)
y = np.linspace(0, x_max, 600)
xg, yg= np.meshgrid(x, y)
idf = interp_df(cart2polNP((xg, yg)))

In [ ]:
df_max-df_min

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data =
    go.Contour(
        z=idf, x= x, y= y,
        contours=dict(
            start= df_min/2, 
            end= df_max,
            size= (df_max-df_min/2)/20
        ),
    ))
fig.update_layout( 
      autosize=False,
      width=800,
      height=600,
)
fig.show()

In [ ]:

e_x = np.sqrt(np.abs(xp))*np.sign(xp)
R, P = np.meshgrid(e_x, th) 
X, Y = R*np.cos(P), R*np.sin(P)

df_cut = df.copy()
df_cut[(df_cut <df_min/5)] = df_min/5

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(rows=1, cols=1,
                    specs=[[{'is_3d': True}]],
                    subplot_titles=['df'],
                    )

#scene1=dict(aspectratio=dict(x=1, y=2, z=0.5),
#                camera_eye=dict(x=-1.00, y=1.0, z=0.2))

fig.add_trace(go.Surface(x=X, y=Y, z=df_cut, colorbar_x=-0.07), 1, 1)
#fig.add_trace(go.Surface(x=X, y=Y, z=Si), 1, 2)
fig.update_scenes( 
                aspectratio=dict(x=2, y=1, z=0.5),
                camera_eye=dict(x=-2.57, y=1.36, z=0.58), row=1, col=1)


fig.update_layout( title_text="df",
      autosize=False,
      width=1000,
      height=900,
  
)
fig.show()
